[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [13]:
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

from sklearn.model_selection import train_test_split

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])



np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )



np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]

features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()

    
X_train_balanced = pd.DataFrame(good_features_X,columns=features_clinic)
X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)

import pickle
f = open('database/nvs_log.text', 'rb')
n_s = pickle.load(f)[1]
f.close()

f = open('database/nvv_log.text', 'rb')
n_v = pickle.load(f)[1]
f.close()

f = open('database/nvf_log.text', 'rb')
n_f = pickle.load(f)[1]
f.close()

f = open('database/svv_log.text', 'rb')
s_v = pickle.load(f)[1]
f.close()

f = open('database/svf_log.text', 'rb')
s_f = pickle.load(f)[1]
f.close()

f = open('database/vvf_log.text', 'rb')
v_f = pickle.load(f)[1]
f.close()


(LogisticRegression(C=50, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None,penalty='l2',random_state=None,
            solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

best for full dataset inter_patient for full class and log 

In [21]:
#lg = LogisticRegression(C=50, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, max_iter=1000, multi_class='auto', n_jobs=None,penalty='l2',random_state=None, solver='liblinear', tol=0.0001, verbose=0, warm_start=False)



lg = LogisticRegression()
lg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [20]:
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
    
#ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

#X_train, y_train=ada.fit_sample(X_train,y_train)
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix 
from sklearn import svm, metrics

print(metrics.confusion_matrix(y_test,y_pred , labels=[0,1,2,3]))
print(classification_report(y_test,y_pred))
print(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[42260   271  1434    56]
 [ 1644   294   106     6]
 [  392    13  2699   116]
 [  253     0    18   117]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     44021
           1       0.51      0.14      0.22      2050
           2       0.63      0.84      0.72      3220
           3       0.40      0.30      0.34       388

   micro avg       0.91      0.91      0.91     49679
   macro avg       0.62      0.56      0.56     49679
weighted avg       0.91      0.91      0.90     49679

0.547945051366379


In [ ]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier

from imblearn.metrics import geometric_mean_score
ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
max_depth = [1,2,4,8,16,32,64,128,236,500,1000]
n_estimators = [1,2,4,8,16,32,64,128,236,500,1000]
criterion = ['gini', 'entropy']

min_weight_fraction_leaf = [0,0.0001,0.001,0.01,0.1]

class_weight = [None,'balanced']
max_features = ['auto', 'sqrt','log2', None, 10, 50, 100, 200]


patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(max_depth)):
    max_d = max_depth[c]
    #print(c_)
    for i in range(len(n_estimators)):
        num = n_estimators[i] 
        for a in range(len(criterion )):
            sov= criterion[a]
            
            for weight in min_weight_fraction_leaf:
                    
                for class_w in class_weight:
                    for features in max_features:
                        svc = RandomForestClassifier(class_weight= class_w, max_depth=max_d,n_estimators=num, criterion=sov, min_weight_fraction_leaf=weight,max_features=features )
            #svc = LogisticRegression(C=c_)
                        model.append(svc)
                        avg = 0
                        scores = []
                        for i in range(0,11):


                            choice = cv[i]

                            X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                            y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                            X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                            y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                            X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                            y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                            patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                            X_train = X_train[patients_ls_1_new!= choice[1]]
                            y_train = y_train[patients_ls_1_new!= choice[1]]



                            from imblearn.under_sampling import RandomUnderSampler

                            ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                            X_train, y_train=ada.fit_sample(X_train,y_train)



                            X_test = np.vstack((X_test_1, X_test_2))
                            y_test = np.hstack((y_test_1, y_test_2))

                            svc.fit(X_train, y_train)
                            y_pred = svc.predict(X_test)
                            scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                                        #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                            count += 1
                            #print('Next')
                        avg = np.mean(scores)
                        accuracies.append(avg)           


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforrandom_interpatinet_true_all_under.txt", "wb" ) )


sort_table(df,1)

In [8]:

### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
 
from imblearn.metrics import geometric_mean_score
ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
#for c in range(len(c_v)):
   # c_ = c_v[c]
    #print(c_)
for i in range(len(class_weight)):
    num = class_weight[i] 
        #for a in range(len(solver )):
            #sov= solver[a]
            #for t in range(len(tol)):
                #to = tol[t]
    #svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
    svc = LogisticRegression(C=50, class_weight= num)
    model.append(svc)
    avg = 0
    scores = []
    for i in range(0,11):
                
                
        choice = cv[i]

        X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
        y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

        X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
        y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




        X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
        y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

        patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


        X_train = X_train[patients_ls_1_new!= choice[1]]
        y_train = y_train[patients_ls_1_new!= choice[1]]





        X_test = np.vstack((X_test_1, X_test_2))
        y_test = np.hstack((y_test_1, y_test_2))

        svc.fit(X_train, y_train)
        y_pred = svc.predict(X_test)
        scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
        count += 1
        avg = np.mean(scores)
    accuracies.append(avg)           


            
df1 = []
        
for score in zip(model,accuracies):
    df1.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
#import pickle

#pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_all.txt", "wb" ) )


sort_table(df1,1)

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' i

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' i

[(LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False), 0.7926738437767732),
 (LogisticRegression(C=50, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='warn', tol=0.0001, verbose=0, warm_start=False),
  0.8088285820057556)]

In [10]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
 
from imblearn.metrics import geometric_mean_score
ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
#for c in range(len(c_v)):
   # c_ = c_v[c]
    #print(c_)
#for i in range(len(class_weight)):
    #num = class_weight[i] 
#for a in range(len(solver )):
    #sov= solver[a]
for t in range(len(tol)):
    to = tol[t]
    #svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
    svc = LogisticRegression(C=50, class_weight= 'balanced',solver='liblinear', tol=to)
    model.append(svc)
    avg = 0
    scores = []
    for i in range(0,11):
                
                
        choice = cv[i]

        X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
        y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

        X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
        y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




        X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
        y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

        patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


        X_train = X_train[patients_ls_1_new!= choice[1]]
        y_train = y_train[patients_ls_1_new!= choice[1]]





        X_test = np.vstack((X_test_1, X_test_2))
        y_test = np.hstack((y_test_1, y_test_2))

        svc.fit(X_train, y_train)
        y_pred = svc.predict(X_test)
        scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
        count += 1
        avg = np.mean(scores)
    accuracies.append(avg)           


            
df2 = []
        
for score in zip(model,accuracies):
    df2.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
#import pickle

#pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_all.txt", "wb" ) )


sort_table(df2,1)

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: Conve

[(LogisticRegression(C=50, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.001, verbose=0, warm_start=False),
  0.8078573535959748),
 (LogisticRegression(C=50, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.01, verbose=0, warm_start=False),
  0.8082628969764966),
 (LogisticRegression(C=50, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=1e-05, verbose=0, warm_start=False),
  0.808437413616618),
 (LogisticRegression(C=50, class_weight='balanced', dual=False,
            fit_interce

In [25]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 

from imblearn.metrics import geometric_mean_score
ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
#ls.extend(['V', 'E'])                         # VEB
#ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
#ls2.extend(["['V']", "['E']"])                         # VEB
#ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(c_v)):
    c_ = c_v[c]
    #print(c_)
    for i in range(len(class_weight)):
        num = class_weight[i] 
        for a in range(len(solver )):
            sov= solver[a]
            for t in range(len(tol)):
                to = tol[t]
                svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
                model.append(svc)
                avg = 0
                scores = []
                for i in range(0,11):
                
                
                    choice = cv[i]

                    X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                    y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                    X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                    y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                    X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                    y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                    patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                    X_train = X_train[patients_ls_1_new!= choice[1]]
                    y_train = y_train[patients_ls_1_new!= choice[1]]

                    from imblearn.under_sampling import RandomUnderSampler
    
                    ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                    X_train, y_train=ada.fit_sample(X_train,y_train)



                    X_test = np.vstack((X_test_1, X_test_2))
                    y_test = np.hstack((y_test_1, y_test_2))

                    svc.fit(X_train, y_train)
                    y_pred = svc.predict(X_test)
                    scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                    count += 1
                avg = np.mean(scores)
                accuracies.append(avg)


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_n_s_under.txt", "wb" ) )


sort_table(df,1)

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

...

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[(LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='newton-cg', tol=0.0001, verbose=0, warm_start=False),
  0.48018276551788436),
 (LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='newton-cg', tol=1e-05, verbose=0, warm_start=False),
  0.4935832213864156),
 (LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
            tol=1e-05, verbose=0, warm_start=False), 0.5000073967861104),
 (LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_interce

In [26]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 

from imblearn.metrics import geometric_mean_score
ls = []
ls.extend(['N', 'L', 'R'])                    # N
#ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
#ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
#ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
#ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(c_v)):
    c_ = c_v[c]
    #print(c_)
    for i in range(len(class_weight)):
        num = class_weight[i] 
        for a in range(len(solver )):
            sov= solver[a]
            for t in range(len(tol)):
                to = tol[t]
                svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
                model.append(svc)
                avg = 0
                scores = []
                for i in range(0,11):
                
                
                    choice = cv[i]

                    X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                    y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                    X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                    y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                    X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                    y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                    patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                    X_train = X_train[patients_ls_1_new!= choice[1]]
                    y_train = y_train[patients_ls_1_new!= choice[1]]

                    from imblearn.under_sampling import RandomUnderSampler
    
                    ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                    X_train, y_train=ada.fit_sample(X_train,y_train)



                    X_test = np.vstack((X_test_1, X_test_2))
                    y_test = np.hstack((y_test_1, y_test_2))

                    svc.fit(X_train, y_train)
                    y_pred = svc.predict(X_test)
                    scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                    count += 1
                avg = np.mean(scores)
                accuracies.append(avg)


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_n_v_uder.txt", "wb" ) )


sort_table(df,1)

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

...

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

[(LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
            tol=0.0001, verbose=0, warm_start=False), 0.8097822145792098),
 (LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
            tol=0.01, verbose=0, warm_start=False), 0.8101100403836896),
 (LogisticRegression(C=5, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.001, verbose=0, warm_start=False),
  0.8111201821775499),
 (LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_sc

In [ ]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 

from imblearn.metrics import geometric_mean_score
ls = []
ls.extend(['N', 'L', 'R'])                    # N
#ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
#ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
#ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
#ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(c_v)):
    c_ = c_v[c]
    #print(c_)
    for i in range(len(class_weight)):
        num = class_weight[i] 
        for a in range(len(solver )):
            sov= solver[a]
            for t in range(len(tol)):
                to = tol[t]
                svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
                model.append(svc)
                avg = 0
                scores = []
                for i in range(0,11):
                
                
                    choice = cv[i]

                    X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                    y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                    X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                    y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                    X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                    y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                    patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                    X_train = X_train[patients_ls_1_new!= choice[1]]
                    y_train = y_train[patients_ls_1_new!= choice[1]]


                    from imblearn.under_sampling import RandomUnderSampler
    
                    ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                    X_train, y_train=ada.fit_sample(X_train,y_train)


                    X_test = np.vstack((X_test_1, X_test_2))
                    y_test = np.hstack((y_test_1, y_test_2))

                    svc.fit(X_train, y_train)
                    y_pred = svc.predict(X_test)
                    
                    scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                    count += 1
                avg = np.mean(scores)
                accuracies.append(avg)


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_n_f_under.txt", "wb" ) )


sort_table(df,1)

In [27]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 

from imblearn.metrics import geometric_mean_score
ls = []
#ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
#ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
#ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
#ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(c_v)):
    c_ = c_v[c]
    #print(c_)
    for i in range(len(class_weight)):
        num = class_weight[i] 
        for a in range(len(solver )):
            sov= solver[a]
            for t in range(len(tol)):
                to = tol[t]
                svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
                model.append(svc)
                avg = 0
                scores = []
                for i in range(0,11):
                
                
                    choice = cv[i]

                    X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                    y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                    X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                    y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                    X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                    y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                    patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                    X_train = X_train[patients_ls_1_new!= choice[1]]
                    y_train = y_train[patients_ls_1_new!= choice[1]]

                    from imblearn.under_sampling import RandomUnderSampler
    
                    ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                    X_train, y_train=ada.fit_sample(X_train,y_train)



                    X_test = np.vstack((X_test_1, X_test_2))
                    y_test = np.hstack((y_test_1, y_test_2))

                    svc.fit(X_train, y_train)
                    y_pred = svc.predict(X_test)
                    scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                    count += 1
                avg = np.mean(scores)
                accuracies.append(avg)


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_s_v_under.txt", "wb" ) )


sort_table(df,1)

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

...

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

[(LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='newton-cg', tol=0.001, verbose=0, warm_start=False),
  0.6423811422271797),
 (LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='lbfgs', tol=1e-05, verbose=0, warm_start=False),
  0.6446744631207431),
 (LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
            tol=0.001, verbose=0, warm_start=False), 0.7052323973257029),
 (LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_intercept

In [28]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 

from imblearn.metrics import geometric_mean_score
ls = []
#.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
#ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
#ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
#ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(c_v)):
    c_ = c_v[c]
    #print(c_)
    for i in range(len(class_weight)):
        num = class_weight[i] 
        for a in range(len(solver )):
            sov= solver[a]
            for t in range(len(tol)):
                to = tol[t]
                svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
                model.append(svc)
                avg = 0
                scores = []
                for i in range(0,11):
                
                
                    choice = cv[i]

                    X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                    y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                    X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                    y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                    X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                    y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                    patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                    X_train = X_train[patients_ls_1_new!= choice[1]]
                    y_train = y_train[patients_ls_1_new!= choice[1]]


                    from imblearn.under_sampling import RandomUnderSampler
    
                    ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                    X_train, y_train=ada.fit_sample(X_train,y_train)


                    X_test = np.vstack((X_test_1, X_test_2))
                    y_test = np.hstack((y_test_1, y_test_2))

                    svc.fit(X_train, y_train)
                    y_pred = svc.predict(X_test)
                    scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                    count += 1
                avg = np.mean(scores)
                accuracies.append(avg)


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_s_f_under.txt", "wb" ) )


sort_table(df,1)

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  w

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergen

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
 

...

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarn

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
 

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)


C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 due to no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samp

[(LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
            tol=1e-05, verbose=0, warm_start=False), 0.1303513354202846),
 (LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
            tol=0.01, verbose=0, warm_start=False), 0.13067014448626463),
 (LogisticRegression(C=5, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.01, verbose=0, warm_start=False),
  0.13413355175524427),
 (LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_sca

In [29]:
### DB1 inter-paitent tuning
from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 

from imblearn.metrics import geometric_mean_score
ls = []
#ls.extend(['N', 'L', 'R'])                    # N
#ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
#patient_ls_1_train = [106,109,112,115,118,122,201,205,207,208,209,215,220,223]
#patient_ls_1_test = [101,108,114,116,119,124,203,230]

#patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]

ls2 = []
#ls2.extend(["['N']","['L']", "['R']"])                    # N
#ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

#np_clinic_1_old_test, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old_test, np_class_ID_2_old_test, patients_ls_1_test, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1_test,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )

from sklearn.linear_model import LogisticRegression




import numpy as np
from sklearn.model_selection import train_test_split
props_8 = np.full((9,1), float(1/9))
props_8 = [float(i) for i in props_8]
patient_ls_1_test = [101,108,114,116,119,124,203,230, 100]
#patient_ls_1_test=np.asarray(patient_ls_1_test)
accuracies = []
model = []
c_v=[0.01,0.1,1,2,5,10, 50, 100]
tol = [0.00001, 0.0001, 0.001, 0.01]
class_weight = [None,'balanced']
solver = ['newton-cg', 'lbfgs', 'liblinear']
patients_ls_1=np.asarray(patients_ls_1)
cv = [[101,106],[108,109],[112,114],[115,116],[118,119],[122,124],[201,203],[205,207],[208,209],[215,220],[223,230]]
count = 1
for c in range(len(c_v)):
    c_ = c_v[c]
    #print(c_)
    for i in range(len(class_weight)):
        num = class_weight[i] 
        for a in range(len(solver )):
            sov= solver[a]
            for t in range(len(tol)):
                to = tol[t]
                svc = LogisticRegression(class_weight= num, solver=sov,tol=to, C=c_)
                model.append(svc)
                avg = 0
                scores = []
                for i in range(0,11):
                
                
                    choice = cv[i]

                    X_test_1 = np_clinic_1_old[patients_ls_1== choice[0]]
                    y_test_1 = np_class_ID_1_old[patients_ls_1== choice[0]]

                    X_test_2 = np_clinic_1_old[patients_ls_1== choice[1]]
                    y_test_2 = np_class_ID_1_old[patients_ls_1== choice[1]]




                    X_train = np_clinic_1_old[patients_ls_1!= choice[0]]
                    y_train = np_class_ID_1_old[patients_ls_1!= choice[0]]

                    patients_ls_1_new = patients_ls_1[patients_ls_1!= choice[0]]


                    X_train = X_train[patients_ls_1_new!= choice[1]]
                    y_train = y_train[patients_ls_1_new!= choice[1]]


                    from imblearn.under_sampling import RandomUnderSampler
    
                    ada=RandomUnderSampler(sampling_strategy='all', replacement=True)

                    X_train, y_train=ada.fit_sample(X_train,y_train)


                    X_test = np.vstack((X_test_1, X_test_2))
                    y_test = np.hstack((y_test_1, y_test_2))

                    svc.fit(X_train, y_train)
                    y_pred = svc.predict(X_test)
                    scores.append(geometric_mean_score(y_test, y_pred, average='weighted'))
                    #scores.append(metric.get_metrics(y_pred,y_test,lb=[0,1,2,3])[2])
                    count += 1
                avg = np.mean(scores)
                accuracies.append(avg)


            
df = []
        
for score in zip(model,accuracies):
    df.append(score)
    

import operator

def sort_table(table, col=0):
    return sorted(table, key=operator.itemgetter(col))
    
import pickle

pickle.dump(sort_table(df,1), open( "database/bestEstimatorforlog_interpatinet_true_v_f_under.txt", "wb" ) )


sort_table(df,1)

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predic

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Converg

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", C

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Converg

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

...

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted sampl

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predic

C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted sampl

C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples.
  warn_for)
C:\Users\chont\Anaconda3\lib\site-packages\imblearn\metrics\_classification.py:241: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predic

[(LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
            tol=0.001, verbose=0, warm_start=False), 0.31120285193748526),
 (LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
            tol=0.001, verbose=0, warm_start=False), 0.31653602942121706),
 (LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
            tol=0.01, verbose=0, warm_start=False), 0.31694879108723717),
 (LogisticRegression(C=0.01, class_weight='balanced', dual=False,
            fit_intercept=True, int

In [2]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
#np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old = DF.get_all_dataframe_patient_specific(200,patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )



from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
ls = []
ls.extend(['N', 'L', 'R'])                    # N
#ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
#ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]


ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
#ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
#ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )
X_train, X_test_n, y_train, y_test_n = train_test_split(np_clinic_1_old, np_class_ID_1_old, test_size=0.80, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)


params =    {
    
    'C':[0.01,0.1,1,2,5,10, 50, 100],
    'tol' : [0.00001, 0.0001, 0.001, 0.01],
    'fit_intercept' : [True, False],
    'class_weight' : [None,'balanced'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
    }




X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]

svc = LogisticRegression(penalty='l2', max_iter=2000)
clf3 = GridSearchCV(svc, params, cv=22)
clf3.fit(X_train, y_train)

sorted(clf3.cv_results_.keys())

import pickle

pickle.dump(clf3, open( "database/bestEstimatorforlog_interpatinet_n_f.txt", "wb" ) )


NameError: name 'train_test_split' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
#np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old = DF.get_all_dataframe_patient_specific(200,patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )



from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
ls = []
#ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
#ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]


ls2 = []
#ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
#ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )
X_train, X_test_n, y_train, y_test_n = train_test_split(np_clinic_1_old, np_class_ID_1_old, test_size=0.80, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)



params =    {
    
    'C':[0.01,0.1,1,2,5,10, 50, 100],
    'tol' : [0.00001, 0.0001, 0.001, 0.01],
    'fit_intercept' : [True, False],
    'class_weight' : [None,'balanced'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
    }




X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]

svc = LogisticRegression(penalty='l2', max_iter=2000)
clf4 = GridSearchCV(svc, params, cv=22)
clf4.fit(X_train, y_train)

sorted(clf4.cv_results_.keys())

import pickle

pickle.dump(clf4, open( "database/bestEstimatorforlog_interpatinet_s_v.txt", "wb" ) )


In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
#np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old = DF.get_all_dataframe_patient_specific(200,patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )



from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
ls = []
#ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
#ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]


ls2 = []
#ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
#ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )
X_train, X_test_n, y_train, y_test_n = train_test_split(np_clinic_1_old, np_class_ID_1_old, test_size=0.80, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)



params =    {
    
    'C':[0.01,0.1,1,2,5,10, 50, 100],
    'tol' : [0.00001, 0.0001, 0.001, 0.01],
    'fit_intercept' : [True, False],
    'class_weight' : [None,'balanced'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
    }



X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]

svc = LogisticRegression(penalty='l2', max_iter=2000)
clf5 = GridSearchCV(svc, params, cv=22)
clf5.fit(X_train, y_train)

sorted(clf5.cv_results_.keys())

import pickle

pickle.dump(clf5, open( "database/bestEstimatorforlog_interpatinet_s_f.txt", "wb" ) )


In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
#np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old = DF.get_all_dataframe_patient_specific(200,patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )



from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
ls = []
#ls.extend(['N', 'L', 'R'])                    # N
#ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]


ls2 = []
#ls2.extend(["['N']","['L']", "['R']"])                    # N
#ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )
X_train, X_test_n, y_train, y_test_n = train_test_split(np_clinic_1_old, np_class_ID_1_old, test_size=0.80, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)



params =    {
    
    'C':[0.01,0.1,1,2,5,10, 50, 100],
    'tol' : [0.00001, 0.0001, 0.001, 0.01],
    'fit_intercept' : [True, False],
    'class_weight' : [None,'balanced'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
    }




X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]

svc = LogisticRegression(penalty='l2', max_iter=2000)
clf6 = GridSearchCV(svc, params, cv=22)
clf6.fit(X_train, y_train)

sorted(clf6.cv_results_.keys())

import pickle

pickle.dump(clf6, open( "database/bestEstimatorforlog_interpatinet_v_f.txt", "wb" ) )


In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
#np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old = DF.get_all_dataframe_patient_specific(200,patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )



from codes.python import load_DF_beats as DF
import numpy as np
from codes.python import metric
from codes.python import features_columns as col
from codes. python import post_process_features_ex as post_features
import pandas as pd
from codes.python import TunedClassifier as classifier
from sklearn.metrics import classification_report, confusion_matrix 
ls = []
ls.extend(['N', 'L', 'R'])                    # N
ls.extend(['A', 'a', 'J', 'S', 'e', 'j'])     # SVEB 
ls.extend(['V', 'E'])                         # VEB
ls.extend(['F'])
#ls.extend([ 'P', '/', 'f', 'u'])
patient_l_1 = [101]
#patient_l_2 = [100]
patient_ls_1 = [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]
patient_ls_2 = [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]


ls2 = []
ls2.extend(["['N']","['L']", "['R']"])                    # N
ls2.extend(["['A']", "['a']", "['J']", "['S']",  "['e']", "['j']"])     # SVEB 
ls2.extend(["['V']", "['E']"])                         # VEB
ls2.extend(["['F']"])
#ls.extend([ "['P']","[ '/']"," ['f']", "['u']"])


good_features_X = np.asarray(pd.read_csv("database/gooddata_X_train_no_outliers.csv").iloc[:,1:263])
good_features_y = np.asarray(pd.read_csv("database/gooddata_y_train_no_outliers.csv").iloc[:,1])


np_clinic_1_old, np_clinic_2_old,np_non_var_1_old, np_non_var_2_old, np_class_ID_1_old, np_class_ID_2_old, patients_ls_1, patients_ls_2, DB1, DB2, DB1_V1, DB2_V1, DB1_non_cli, DB2_non_cli, DB1_dwt, DB2_dwt, DB1_dwt_V1, DB2_dwt_V1 = DF.get_all_dataframe(patient_l_1=patient_ls_1,patient_l_2=patient_ls_2 , ls=ls, ls2=ls2 )
X_train, X_test_n, y_train, y_test_n = train_test_split(np_clinic_1_old, np_class_ID_1_old, test_size=0.80, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)



params =    {
    
    'C':[0.01,0.1,1,2,5,10, 50, 100],
    'tol' : [0.00001, 0.0001, 0.001, 0.01],
    'fit_intercept' : [True, False],
    'class_weight' : [None,'balanced'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
    }




X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]

svc = LogisticRegression(penalty='l2', max_iter=2000)
clf7 = GridSearchCV(svc, params, cv=22)
clf7.fit(X_train, y_train)

sorted(clf7.cv_results_.keys())


import pickle

pickle.dump(clf7, open( "database/bestEstimatorforlog_interpatinet_all.txt", "wb" ) )


In [ ]:
## Train test from DB1

np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 0] = 1
y_train[y_train == 0] = 0

y_test[y_test != 0] = 1
y_test[y_test == 0] = 0



"""
print("svm_linear")
print("all")
classifier.svm_model_linear(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.svm_model_linear(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.svm_model_linear(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.svm_model_linear(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.svm_model_linear(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.svm_model_linear(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.svm_model_linear(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


print("svm_poly")
print("all")
classifier.svm_model_poly(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.svm_model_poly(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.svm_model_poly(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.svm_model_poly(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.svm_model_poly(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.svm_model_poly(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.svm_model_poly(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])






print("xgboost")
print("all")
classifier.xgboost(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.xgboost(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.xgboost(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.xgboost(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.xgboost(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.xgboost(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.xgboost(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])




print("ada")
classifier.ada(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.ada(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.ada(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.ada(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.ada(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.ada(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.ada(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


"""



print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
## Train test from DB1
## Train test from DB1

np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 1] = 0
y_train[y_train == 1] = 1

y_test[y_test != 1] = 0
y_test[y_test == 1] = 1




print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
## Train test from DB1

np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 2] = 0
y_train[y_train == 2] = 1

y_test[y_test != 2] = 0
y_test[y_test == 2] = 1






print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
## Train test from DB1

np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 3] = 0
y_train[y_train == 3] = 1

y_test[y_test != 3] = 0
y_test[y_test == 3] = 1






print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test = X_test[2000:4000]
y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 0] = 1
y_train[y_train == 0] = 0

y_test[y_test != 0] = 1
y_test[y_test == 0] = 0







print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test = X_test[2000:4000]
y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 1] = 0
y_train[y_train == 1] = 1

y_test[y_test != 1] = 0
y_test[y_test == 1] = 1







print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test = X_test[2000:4000]
y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 2] = 0
y_train[y_train == 2] = 1

y_test[y_test != 2] = 0
y_test[y_test == 2] = 1







print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


from sklearn.model_selection import train_test_split

X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test = X_test[2000:4000]
y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 3] = 0
y_train[y_train == 3] = 1

y_test[y_test != 3] = 0
y_test[y_test == 3] = 1






print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


#from sklearn.model_selection import train_test_split

#X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 0] =1
y_train[y_train == 0] = 0

y_test[y_test != 0] = 1
y_test[y_test == 0] = 0






print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


#from sklearn.model_selection import train_test_split

#X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 1] = 0
y_train[y_train == 1] = 1

y_test[y_test != 1] = 0
y_test[y_test == 1] = 1







print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


#from sklearn.model_selection import train_test_split

#X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 2] = 0
y_train[y_train == 2] = 1

y_test[y_test != 2] = 0
y_test[y_test == 2] = 1






print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])


In [ ]:
np_class_ID_1_old = [int(i) for i in np_class_ID_1_old]
np_class_ID_2_old = [int(i) for i in np_class_ID_2_old]
X_train = np_clinic_1_old
X_test = np_clinic_2_old
y_train = np.asarray(np_class_ID_1_old)
y_test = np.asarray(np_class_ID_2_old)
input_size=X_train.shape[1]


features_clinic,c_ID,f_M, f_V, f_d , norm_mlii, norm_v1 , norm_dtw = col.get_columns()


#from sklearn.model_selection import train_test_split

#X_train, test, y_train, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#X_test = X_test[2000:4000]
#y_test = y_test[2000:4000]
#train, X_test, ytrain, y_test = train_test_split(X_test, y_test, test_size=0.80, random_state=42)

X_train = pd.DataFrame(X_train,columns=features_clinic)
X_test = pd.DataFrame(X_test,columns=features_clinic)


y_train[y_train != 3] = 0
y_train[y_train == 3] = 1

y_test[y_test != 3] = 0
y_test[y_test == 3] = 1




print("Log")
print("all")
#classifier.logisticRegress(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.logisticRegress(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.logisticRegress(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.logisticRegress(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.logisticRegress(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.logisticRegress(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.logisticRegress(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])



print("Lin")
print("all")
#classifier.Linear_D(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.Linear_D(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.Linear_D(X_train[n_v], y_train, X_test[n_v], y_test, jk=False ,labels=[0,1,2,3])
print("n_f")
classifier.Linear_D(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.Linear_D(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.Linear_D(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.Linear_D(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])

print("randomforest")
print("all")
#classifier.randomForest(X_train[feature_good], y_train, X_test[feature_good], y_test, jk=False,labels=[0,1,2,3])
print("n_s")
classifier.randomForest(X_train[n_s], y_train, X_test[n_s], y_test, jk=False,labels=[0,1,2,3])
print("n_v")
classifier.randomForest(X_train[n_v], y_train, X_test[n_v], y_test, jk=False,labels=[0,1,2,3])
print("n_f")
classifier.randomForest(X_train[n_f], y_train, X_test[n_f], y_test, jk=False,labels=[0,1,2,3])
print("s_v")
classifier.randomForest(X_train[s_v], y_train, X_test[s_v], y_test, jk=False,labels=[0,1,2,3])
print("s_f")
classifier.randomForest(X_train[s_f], y_train, X_test[s_f], y_test, jk=False,labels=[0,1,2,3])
print("v_f")
classifier.randomForest(X_train[v_f], y_train, X_test[v_f], y_test, jk=False,labels=[0,1,2,3])
